# 아파트 실거래가 예측 - https://bit.ly/3v5edhJ

### 부동산 빅데이터와 AI를 이용하여 실거래가를 예측 분석 | 회귀 | RMSE

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
print(train_df.shape)
train_df.head()

(1216553, 13)


,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500
1,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.17,1973,200801,1~10,6,20000
2,2,3578,서울특별시,필운동,174-1,두레엘리시안,필운동 174-1 두레엘리시안,84.74,2007,200801,1~10,6,38500
3,3,10957,서울특별시,내수동,95,파크팰리스,내수동 95 파크팰리스,146.39,2003,200801,11~20,15,118000
4,4,10639,서울특별시,내수동,110-15,킹스매너,내수동 110-15 킹스매너,194.43,2004,200801,21~31,3,120000


In [3]:
# 데이터 줄이기

train_df = train_df.loc[train_df["transaction_id"]%100==0, :]
train_df = train_df.reset_index(drop=True)
train_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500
1,100,3926,서울특별시,수송동,85,로얄팰리스스위트,수송동 85 로얄팰리스스위트,40.07,2005,200803,11~20,10,19000
2,200,10147,서울특별시,창신동,703,창신쌍용2,창신동 703 창신쌍용2,106.62,1993,200804,11~20,5,43500
3,300,10146,서울특별시,창신동,702,창신쌍용1,창신동 702 창신쌍용1,106.62,1992,200806,1~10,1,41500
4,400,7093,서울특별시,숭인동,569-2,숭인동 104동 (569-2),숭인동 569-2 숭인동 104동 (569-2),59.74,2000,200809,1~10,7,17700


In [4]:
print(test_df.shape)
test_df.head()

(5463, 12)


,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor
0,1145756,10453,서울특별시,목동,938,청학,목동 938 청학,35.5500,2002,201711,11~20,2
1,1198704,989,부산광역시,초량동,1143-8,고관맨션,초량동 1143-8 고관맨션,68.7200,1977,201708,21~31,2
2,1222384,8597,부산광역시,괴정동,447-13,우림그린,괴정동 447-13 우림그린,72.5400,1989,201710,11~20,2
3,1179897,11086,서울특별시,대치동,1007-2,풍림아이원4차(1007-2),대치동 1007-2 풍림아이원4차(1007-2),111.5400,2004,201707,1~10,10
4,1223091,2121,부산광역시,다대동,1670,다대롯데캐슬블루,다대동 1670 다대롯데캐슬블루,119.6398,2014,201712,11~20,21


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12159 entries, 0 to 12158
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          12159 non-null  int64  
 1   apartment_id            12159 non-null  int64  
 2   city                    12159 non-null  object 
 3   dong                    12159 non-null  object 
 4   jibun                   12159 non-null  object 
 5   apt                     12159 non-null  object 
 6   addr_kr                 12159 non-null  object 
 7   exclusive_use_area      12159 non-null  float64
 8   year_of_completion      12159 non-null  int64  
 9   transaction_year_month  12159 non-null  int64  
 10  transaction_date        12159 non-null  object 
 11  floor                   12159 non-null  int64  
 12  transaction_real_price  12159 non-null  int64  
dtypes: float64(1), int64(6), object(6)
memory usage: 1.2+ MB


In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5463 entries, 0 to 5462
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          5463 non-null   int64  
 1   apartment_id            5463 non-null   int64  
 2   city                    5463 non-null   object 
 3   dong                    5463 non-null   object 
 4   jibun                   5463 non-null   object 
 5   apt                     5463 non-null   object 
 6   addr_kr                 5463 non-null   object 
 7   exclusive_use_area      5463 non-null   float64
 8   year_of_completion      5463 non-null   int64  
 9   transaction_year_month  5463 non-null   int64  
 10  transaction_date        5463 non-null   object 
 11  floor                   5463 non-null   int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 512.3+ KB


## city

In [7]:
train_df["city"].unique()

array(['서울특별시', '부산광역시'], dtype=object)

In [8]:
test_df["city"].unique()

array(['서울특별시', '부산광역시'], dtype=object)

### Labeling

In [9]:
train_df.loc[train_df["city"] == "서울특별시", "city"] = 0
train_df.loc[train_df["city"] == "부산광역시", "city"] = 1
train_df["city"] = train_df["city"].astype('int')

test_df.loc[test_df["city"] == "서울특별시", "city"] = 0
test_df.loc[test_df["city"] == "부산광역시", "city"] = 1
test_df["city"] = test_df["city"].astype('int')

In [10]:
train_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,0,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500
1,100,3926,0,수송동,85,로얄팰리스스위트,수송동 85 로얄팰리스스위트,40.07,2005,200803,11~20,10,19000
2,200,10147,0,창신동,703,창신쌍용2,창신동 703 창신쌍용2,106.62,1993,200804,11~20,5,43500
3,300,10146,0,창신동,702,창신쌍용1,창신동 702 창신쌍용1,106.62,1992,200806,1~10,1,41500
4,400,7093,0,숭인동,569-2,숭인동 104동 (569-2),숭인동 569-2 숭인동 104동 (569-2),59.74,2000,200809,1~10,7,17700


In [11]:
test_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor
0,1145756,10453,0,목동,938,청학,목동 938 청학,35.5500,2002,201711,11~20,2
1,1198704,989,1,초량동,1143-8,고관맨션,초량동 1143-8 고관맨션,68.7200,1977,201708,21~31,2
2,1222384,8597,1,괴정동,447-13,우림그린,괴정동 447-13 우림그린,72.5400,1989,201710,11~20,2
3,1179897,11086,0,대치동,1007-2,풍림아이원4차(1007-2),대치동 1007-2 풍림아이원4차(1007-2),111.5400,2004,201707,1~10,10
4,1223091,2121,1,다대동,1670,다대롯데캐슬블루,다대동 1670 다대롯데캐슬블루,119.6398,2014,201712,11~20,21


## drop addr_kr

In [12]:
train_df.drop('addr_kr', axis=1, inplace=True)
test_df.drop('addr_kr', axis=1, inplace=True)

## 한강이 보이는 동네

In [13]:
hanriver_dong = ['성수동1가','삼성동','이촌동','공덕동','서교동','한강로3가','목동','고덕동', '암사동', '천호동', '풍납동',
                '잠실동', '청담동', '압구정동', '신사동', '잠원동', '반포동', '흑석동', '노량진동', '여의동', '당산동',
                '양평동', '염창동', '가양동', '방화동']

# isin을 사용해서 한강이 보이는 동네는 1, 아니면 0
# True, False를 integer로 변환

train_df['hanriver'] =
test_df['hanriver'] =

## 평균 실거래가로 동네 sorting

In [14]:
# "dong"으로 groupby 후, 평균값으로 집계
# 실거래가를 기준으로 오름차순 정렬 후 reset_index()
dong = 

# dong의 인덱스를 idx 컬럼에 추가
dong["idx"] = 
dong

,dong,transaction_real_price,idx
0,봉래동5가,5500.000000,0
1,일광면 삼성리,7600.000000,1
2,영주동,8481.250000,2
3,토성동2가,9382.666667,3
4,대청동4가,9500.000000,4
...,...,...,...
378,동빙고동,123000.000000,378
379,서빙고동,124416.666667,379
380,반포동,133264.058140,380
381,압구정동,154906.756757,381


In [15]:
# dong_rank에 동 이름을 key, 순서를 value
dong_rank = {}
for i in range(len(dong)):
    
dong_rank

{'봉래동5가': 0,
 '일광면 삼성리': 1,
 '영주동': 2,
 '토성동2가': 3,
 '대청동4가': 4,
 '대청동1가': 5,
 '기장읍 서부리': 6,
 '일광면 이천리': 7,
 '동삼동': 8,
 '감만동': 9,
 '동대신동1가': 10,
 '미근동': 11,
 '반송동': 12,
 '구평동': 13,
 '수정동': 14,
 '아미동2가': 15,
 '서동': 16,
 '학장동': 17,
 '부용동1가': 18,
 '봉래동4가': 19,
 '기장읍 청강리': 20,
 '영등포동3가': 21,
 '모라동': 22,
 '덕천동': 23,
 '보수동1가': 24,
 '덕포동': 25,
 '장림동': 26,
 '금사동': 27,
 '남산동': 28,
 '기장읍 동부리': 29,
 '명장동': 30,
 '기장읍 대변리': 31,
 '영선동3가': 32,
 '금곡동': 33,
 '우암동': 34,
 '신평동': 35,
 '구포동': 36,
 '전포동': 37,
 '기장읍 대라리': 38,
 '당감동': 39,
 '대창동1가': 40,
 '다대동': 41,
 '삼락동': 42,
 '서대신동1가': 43,
 '괴정동': 44,
 '기장읍 교리': 45,
 '보수동2가': 46,
 '좌천동': 47,
 '감천동': 48,
 '초량동': 49,
 '개금동': 50,
 '암남동': 51,
 '만덕동': 52,
 '주례동': 53,
 '동대신동3가': 54,
 '지사동': 55,
 '흥인동': 56,
 '충무동2가': 57,
 '당리동': 58,
 '토성동5가': 59,
 '정관읍 방곡리': 60,
 '봉래동1가': 61,
 '청학동': 62,
 '가야동': 63,
 '엄궁동': 64,
 '청룡동': 65,
 '신창동1가': 66,
 '하단동': 67,
 '광안동': 68,
 '수안동': 69,
 '초읍동': 70,
 '범천동': 71,
 '남항동3가': 72,
 '범전동': 73,
 '토성동1가': 74,
 '남항동1가': 75,
 '대평동1가': 76,
 '

In [16]:
# df의 dong_rank에 위 dong_rank 딕셔너리 매핑
train_df["dong_rank"] = 
test_df["dong_rank"] = 

# dong 컬럼 drop


train_df.head()

,transaction_id,apartment_id,city,jibun,apt,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,hanriver,dong_rank
0,0,7622,0,6-13,신현(101동),84.82,2002,200801,21~31,2,37500,0,223
1,100,3926,0,85,로얄팰리스스위트,40.07,2005,200803,11~20,10,19000,0,111
2,200,10147,0,703,창신쌍용2,106.62,1993,200804,11~20,5,43500,0,220
3,300,10146,0,702,창신쌍용1,106.62,1992,200806,1~10,1,41500,0,220
4,400,7093,0,569-2,숭인동 104동 (569-2),59.74,2000,200809,1~10,7,17700,0,90


## 평균 실거래가로 아파트 sorting

In [17]:
apt = 
apt["idx"] = 


apt_rank = {}
for i in range(len(apt)):
    

train_df["apt_rank"] = 
test_df["apt_rank"] = 

train_df.drop()
test_df.drop()

train_df.head()

,transaction_id,apartment_id,city,jibun,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,hanriver,dong_rank,apt_rank
0,0,7622,0,6-13,84.82,2002,200801,21~31,2,37500,0,223,2309
1,100,3926,0,85,40.07,2005,200803,11~20,10,19000,0,111,1365
2,200,10147,0,703,106.62,1993,200804,11~20,5,43500,0,220,2028
3,300,10146,0,702,106.62,1992,200806,1~10,1,41500,0,220,2306
4,400,7093,0,569-2,59.74,2000,200809,1~10,7,17700,0,90,856


## 평균 실거래가로 지번 sorting

In [18]:
jibun = 
jibun["idx"] = 


jibun_rank = {}
for i in range(len(jibun)):
    
    
train_df["jibun_rank"] = 
test_df["jibun_rank"] = 

train_df.drop()
test_df.drop()

train_df.head()

,transaction_id,apartment_id,city,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,hanriver,dong_rank,apt_rank,jibun_rank
0,0,7622,0,84.82,2002,200801,21~31,2,37500,0,223,2309,2146
1,100,3926,0,40.07,2005,200803,11~20,10,19000,0,111,1365,910
2,200,10147,0,106.62,1993,200804,11~20,5,43500,0,220,2028,1885
3,300,10146,0,106.62,1992,200806,1~10,1,41500,0,220,2306,1748
4,400,7093,0,59.74,2000,200809,1~10,7,17700,0,90,856,785


In [19]:
train_df.head()

,transaction_id,apartment_id,city,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,hanriver,dong_rank,apt_rank,jibun_rank
0,0,7622,0,84.82,2002,200801,21~31,2,37500,0,223,2309,2146
1,100,3926,0,40.07,2005,200803,11~20,10,19000,0,111,1365,910
2,200,10147,0,106.62,1993,200804,11~20,5,43500,0,220,2028,1885
3,300,10146,0,106.62,1992,200806,1~10,1,41500,0,220,2306,1748
4,400,7093,0,59.74,2000,200809,1~10,7,17700,0,90,856,785


## 거래 날짜 Labeling

In [20]:
train_df['transaction_date'].unique(), test_df['transaction_date'].unique()

(array(['21~31', '11~20', '1~10', '21~30', '21~28', '21~29'], dtype=object),
 array(['11~20', '21~31', '1~10', '21~30', '21~28'], dtype=object))

In [21]:
date_range = {'1~10': 0, '11~20': 1, '21~28': 3, '21~29': 3, '21~30': 3, '21~31': 3}

train_df.replace({"transaction_date": date_range}, inplace=True)
test_df.replace({"transaction_date": date_range}, inplace=True)

train_df.head()

,transaction_id,apartment_id,city,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price,hanriver,dong_rank,apt_rank,jibun_rank
0,0,7622,0,84.82,2002,200801,3,2,37500,0,223,2309,2146
1,100,3926,0,40.07,2005,200803,1,10,19000,0,111,1365,910
2,200,10147,0,106.62,1993,200804,1,5,43500,0,220,2028,1885
3,300,10146,0,106.62,1992,200806,0,1,41500,0,220,2306,1748
4,400,7093,0,59.74,2000,200809,0,7,17700,0,90,856,785


In [22]:
train_df.info(), test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12159 entries, 0 to 12158
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          12159 non-null  int64  
 1   apartment_id            12159 non-null  int64  
 2   city                    12159 non-null  int64  
 3   exclusive_use_area      12159 non-null  float64
 4   year_of_completion      12159 non-null  int64  
 5   transaction_year_month  12159 non-null  int64  
 6   transaction_date        12159 non-null  int64  
 7   floor                   12159 non-null  int64  
 8   transaction_real_price  12159 non-null  int64  
 9   hanriver                12159 non-null  int64  
 10  dong_rank               12159 non-null  int64  
 11  apt_rank                12159 non-null  int64  
 12  jibun_rank              12159 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 1.2 MB
<class 'pandas.core.frame.DataFrame'>
RangeIn

(None, None)

## test_df null 값 처리

In [23]:
test_df["dong_rank"]
test_df["apt_rank"]
test_df["jibun_rank"]

## 데이터셋 분리

In [24]:
from sklearn.model_selection import train_test_split

x = 
y = 



print(x_train.shape, x_valid.shape)
print(y_train.shape, y_valid.shape)

(8511, 12) (3648, 12)
(8511,) (3648,)


## 모델 학습 및 예측

### SVM

In [25]:
from sklearn.svm import SVC

svc = 
svc.

y_hat = svc.

In [26]:
from sklearn.metrics import mean_squared_error

svm_rmse = mean_squared_error(y_valid, y_hat)**0.5
svm_rmse

31341.073668229004

In [ ]:
y_test = 

In [ ]:
svm_submission = pd.concat([test_df['transaction_id'], y_test], axis=1)

svm_submission.columns = ['transaction_id', 'transaction_real_price']
svm_submission.to_csv('svm_submission.csv', header = True, index = False)

In [ ]:
test = pd.read_csv('svm_submission.csv')
test

In [ ]:
test.transaction_real_price.unique()

### RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = 
rf.


y_hat = rf.

In [ ]:
rf_rmse = mean_squared_error(y_valid, y_hat)**0.5
rf_rmse

In [ ]:
y_test = 

In [ ]:
rf_submission = pd.concat([test_df['transaction_id'], y_test], axis=1)


rf_submission.columns = ['transaction_id', 'transaction_real_price']
rf_submission.to_csv('rf_submission.csv', header = True, index = False)

In [ ]:
test = pd.read_csv('rf_submission.csv')
test